# 3D Boxes (APC renderer)

Render APC-style 3D cubes using the built-in `RenderModule`.
Provide your `abstract_scene_dict` and get a rendered PIL image.


In [1]:
import os
import json
from tqdm import tqdm

In [2]:
# Force reload the renderer module to ensure we get our changes
import importlib
import apc.renderer
importlib.reload(apc.renderer)
print("Reloaded renderer module")

Reloaded renderer module


In [3]:
import os
# Configure headless rendering environment
os.environ['PYGLET_HEADLESS'] = '1'
os.environ['PYOPENGL_PLATFORM'] = 'osmesa'  # Use OSMesa for software rendering
os.environ['DISPLAY'] = ':0'  # Fallback display

# Try importing key components to diagnose what's available
try:
    import OpenGL
    print("OpenGL version:", OpenGL.__version__)
    from OpenGL import GL
    print("GL import ok")
    import pyglet
    print("pyglet version info:", pyglet.version)  # Use pyglet.version instead of __version__
    # Force software rendering
    pyglet.options['shadow_window'] = False
    pyglet.options['debug_gl'] = False
except ImportError as e:
    print("Import failed:", str(e))
    print("Try: pip install 'pyglet==1.5.27' PyOpenGL PyOpenGL-accelerate")

OpenGL version: 3.1.10
GL import ok
pyglet version info: 1.5.27


In [4]:
import numpy as np
from IPython.display import display
# RenderModule (local renderer)
from apc.renderer import RenderModule

## Define your abstract scene
Positions are in 3D (x, y, z), orientation is a 3D direction vector.
The 'camera' entry is preserved but not drawn.


In [5]:
with open("rl_data/orien3.json", "r") as f:
    scene_list = json.load(f)

In [6]:
abstract_scene_dict = {
    "camera": {"position": [0, 0, 0], "orientation": [0, 0, 1]},
    "obj_a": {"position": [-1.0, 1.0, -3.0], "orientation": [0.0, 1.0, 0.0]},
    "obj_b": {"position": [2.0, -1.5, -6.0], "orientation": [1.0, 0.0, 0.0]}
  }

## Convert to renderer format + draw
APC renderer expects TriMesh convention and a color per object.


In [7]:
color_dict = {
    'blue':   [0, 0, 1],
    'green':  [0, 1, 0],
    'yellow': [1, 1, 0],
    'purple': [1, 0, 1],
    'orange': [1, 0.5, 0],
    'brown':  [0.5, 0.25, 0],
    'gray':   [0.5, 0.5, 0.5],
}


In [8]:
def to_trimesh_scene(scene: dict, ref_viewer: str = 'camera') -> dict:
    out = {}
    color_idx = 0  # Separate counter for non-camera objects
    
    for name, entry in scene.items():
        # print(name)
        if name == ref_viewer or not name.startswith("object"):
            continue  # Skip camera object

        pos = np.array(entry['position']).copy()
        ori = np.array(entry['orientation']).copy()
        
        # Normalize orientation before conversion
        # ori = ori / np.linalg.norm(ori)

        color = entry['color']
        
        # # Copy colors with proper indexing
        # print(color)
        color = color_dict[str(color)]
        # color_idx += 1
        
        out[name] = {
            'position': pos,
            'orientation': ori,
            'color': color,
            'original_pos': np.array(entry['position']),
            'original_ori': ori
        }
    return out

# renderer = RenderModule(device='cpu')
# # scene = renderer.recenter_camera(abstract_scene_dict)
# scene_for_renderer = to_trimesh_scene(abstract_scene_dict, ref_viewer='camera')

# # Debug print converted scene
# print("\nConverted scene for renderer:")
# for name, entry in scene_for_renderer.items():
    
#     print(f"  {name}:")
#     print(f"    position: {entry['position']}")
#     print(f"    orientation: {entry['orientation']}")
#     print(f"    color: {entry['color']}")

# img = renderer.render_visual_prompt(
#     scene_for_renderer,
#     ref_viewer='camera',
#     trace_save_dir='outputs',
#     render_whole_scene=True,
#     visualize_trace=True,
# )

In [9]:
# img

In [10]:
for i, scene_data in tqdm(enumerate(scene_list)):

    # Convert lists to numpy arrays
    abstract_scene_dict = {
        key: {
            k: np.array(v)
            for k, v in value.items()
        }
        for key, value in scene_data.items() if key == 'camera' or key.startswith("object")
    }

    # print(f"\n=== Rendering Scene {i} ===")
    # print(abstract_scene_dict)

    scene_for_renderer = to_trimesh_scene(abstract_scene_dict, ref_viewer='camera')

    # print(scene_for_renderer)

    # Example: render the scene
    renderer = RenderModule(device='cpu')
    img = renderer.render_visual_prompt(
        scene_for_renderer,
        ref_viewer='camera',
        trace_save_dir=f"/ocean/projects/cis250208p/shared/datasets/synthetic/orien/scene_{i+200}",
        render_whole_scene=True,
        visualize_trace=True,
    )

    
    scene_list[i]["img"] =  f"/ocean/projects/cis250208p/shared/datasets/synthetic/orien/scene_{i+200}/visual_prompt.png"

    

    # img.show()

100it [01:42,  1.03s/it]


In [11]:
file = "/jet/home/vwei/Ego2Allo/rl_data/orien3.json"
with open(file, 'w') as f:
    json.dump(scene_list, f, indent = 2)